In [1]:
# Import required libraries

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [2]:
# Import file 

df_fake = pd.read_csv("Fake.csv")
df_true = pd.read_csv("True.csv")

In [3]:
# Set display to show more from cell

pd.set_option('display.max_colwidth', None)

In [4]:
df_fake

title  \
0                  Donald Trump Sends Out Embarrassing New Year’s Eve Message; This is Disturbing   
1                            Drunk Bragging Trump Staffer Started Russian Collusion Investigation   
2       Sheriff David Clarke Becomes An Internet Joke For Threatening To Poke People ‘In The Eye’   
3                   Trump Is So Obsessed He Even Has Obama’s Name Coded Into His Website (IMAGES)   
4                           Pope Francis Just Called Out Donald Trump During His Christmas Speech   
...                                                                                           ...   
23476                               McPain: John McCain Furious That Iran Treated US Sailors Well   
23477           JUSTICE? Yahoo Settles E-mail Privacy Class-action: $4M for Lawyers, $0 for Users   
23478       Sunnistan: US and Allied ‘Safe Zone’ Plan to Take Territorial Booty in Northern Syria   
23479                         How to Blow $700 Million: Al Jazeera America Finally Calls it Quits   
23480           10 U.S. Navy Sailors Held by Iranian Military – Signs of a Neocon Political Stunt   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [5]:
df_true

title  \
0           As U.S. budget fight looms, Republicans flip their fiscal script   
1           U.S. military to accept transgender recruits on Monday: Pentagon   
2               Senior U.S. Republican senator: 'Let Mr. Mueller do his job'   
3                FBI Russia probe helped by Australian diplomat tip-off: NYT   
4      Trump wants Postal Service to charge 'much more' for Amazon shipments   
...                                                                      ...   
21412          'Fully committed' NATO backs new U.S. approach on Afghanistan   
21413                   LexisNexis withdrew two products from Chinese market   
21414                      Minsk cultural hub becomes haven from authorities   
21415          Vatican upbeat on possibility of Pope Francis visiting Russia   
21416                   Indonesia to buy $1.14 billion worth of Russian jets   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

Normalise data as there are many different symbols and fonts to reduce the complexity, and allow model to learn better.

In [6]:
# To Lower text case & remove punctuations except underscore
# Reference from:
# https://medium.com/data-science-in-your-pocket/text-classification-for-beginners-in-nlp-with-codes-93c94a8b9ec0

import re

df_fake['title'] = df_fake['title'].transform(lambda value:re.sub(r'[^\w\s]','',value.lower()))
df_true['title'] = df_true['title'].transform(lambda value:re.sub(r'[^\w\s]','',value.lower()))

df_fake['text'] = df_fake['text'].transform(lambda value:re.sub(r'[^\w\s]','',value.lower()))
df_true['text'] = df_true['text'].transform(lambda value:re.sub(r'[^\w\s]','',value.lower()))

df_fake['subject'] = df_fake['subject'].transform(lambda value:re.sub(r'[^\w\s]','',value.lower()))
df_true['subject'] = df_true['subject'].transform(lambda value:re.sub(r'[^\w\s]','',value.lower()))

# Only change to lowercase for date without removing punctuations as its date format
df_fake['date'] = df_fake['date'].transform(lambda value:value.lower())
df_true['date'] = df_true['date'].transform(lambda value:value.lower())

In [7]:
df_fake.head()

,title,text,subject,date
0,donald trump sends out embarrassing new years eve message this is disturbing,donald trump just couldn t wish all americans a happy new year and leave it at that instead he had to give a shout out to his enemies haters and the very dishonest fake news media the former reality show star had just one job to do and he couldn t do it as our country rapidly grows stronger and smarter i want to wish all of my friends supporters enemies haters and even the very dishonest fake news media a happy and healthy new year president angry pants tweeted 2018 will be a great year for america as our country rapidly grows stronger and smarter i want to wish all of my friends supporters enemies haters and even the very dishonest fake news media a happy and healthy new year 2018 will be a great year for america donald j trump realdonaldtrump december 31 2017trump s tweet went down about as welll as you d expectwhat kind of president sends a new year s greeting like this despicable petty infantile gibberish only trump his lack of decency won t even allow him to rise above the gutter long enough to wish the american citizens a happy new year bishop talbert swan talbertswan december 31 2017no one likes you calvin calvinstowell december 31 2017your impeachment would make 2018 a great year for america but i ll also accept regaining control of congress miranda yaver mirandayaver december 31 2017do you hear yourself talk when you have to include that many people that hate you you have to wonder why do the they all hate me alan sandoval alansandoval13 december 31 2017who uses the word haters in a new years wish marlene marlene399 december 31 2017you can t just say happy new year koren pollitt korencarpenter december 31 2017here s trump s new year s eve tweet from 2016happy new year to all including to my many enemies and those who have fought me and lost so badly they just don t know what to do love donald j trump realdonaldtrump december 31 2016this is nothing new for trump he s been doing this for yearstrump has directed messages to his enemies and haters for new year s easter thanksgiving and the anniversary of 911 pictwittercom4fpae2kypa daniel dale ddale8 december 31 2017trump s holiday tweets are clearly not presidentialhow long did he work at hallmark before becoming president steven goodine sgoodine december 31 2017he s always been like this the only difference is that in the last few years his filter has been breaking down roy schulze thbthttt december 31 2017who apart from a teenager uses the term haters wendy wendywhistles december 31 2017he s a fucking 5 year old who knows rainyday80 december 31 2017so to all the people who voted for this a hole thinking he would change once he got into power you were wrong 70yearold men don t change and now he s a year olderphoto by andrew burtongetty images,news,"december 31, 2017"
1,drunk bragging trump staffer started russian collusion investigation,house intelligence committee chairman devin nunes is going to have a bad day he s been under the assumption like many of us that the christopher steeledossier was what prompted the russia investigation so he s been lashing out at the department of justice and the fbi in order to protect trump as it happens the dossier is not what started the investigation according to documents obtained by the new york timesformer trump campaign adviser george papadopoulos was drunk in a wine bar when he revealed knowledge of russian opposition research on hillary clintonon top of that papadopoulos wasn t just a covfefe boy for trump as his administration has alleged he had a much larger role but none so damning as being a drunken fool in a wine bar coffee boys don t help to arrange a new york meeting between trump and president abdel fattah elsisi of egypt two months before the election it was known before that the former aide set up meetings with world leaders for trump but team trump ran with him being merely a coffee boyin may 2016 papadopoulos rev

In [8]:
df_true.head()

title  \
0        as us budget fight looms republicans flip their fiscal script   
1        us military to accept transgender recruits on monday pentagon   
2               senior us republican senator let mr mueller do his job   
3            fbi russia probe helped by australian diplomat tipoff nyt   
4  trump wants postal service to charge much more for amazon shipments   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [9]:
# Adding new column to True/Fake column with 'fake' value for df_fake dataframe and 
# 'true' value in True/Fake column for df_true dataframe

df_fake['True/Fake'] = 'fake'
df_true['True/Fake'] = 'true'

In [10]:
# Combine both dataframe of fake and true into one dataframe

combined_df = df_fake.append(df_true).reset_index(drop=True)

In [11]:
combined_df

,title,text,subject,date,True/Fake
0,donald trump sends out embarrassing new years eve message this is disturbing,donald trump just couldn t wish all americans a happy new year and leave it at that instead he had to give a shout out to his enemies haters and the very dishonest fake news media the former reality show star had just one job to do and he couldn t do it as our country rapidly grows stronger and smarter i want to wish all of my friends supporters enemies haters and even the very dishonest fake news media a happy and healthy new year president angry pants tweeted 2018 will be a great year for america as our country rapidly grows stronger and smarter i want to wish all of my friends supporters enemies haters and even the very dishonest fake news media a happy and healthy new year 2018 will be a great year for america donald j trump realdonaldtrump december 31 2017trump s tweet went down about as welll as you d expectwhat kind of president sends a new year s greeting like this despicable petty infantile gibberish only trump his lack of decency won t even allow him to rise above the gutter long enough to wish the american citizens a happy new year bishop talbert swan talbertswan december 31 2017no one likes you calvin calvinstowell december 31 2017your impeachment would make 2018 a great year for america but i ll also accept regaining control of congress miranda yaver mirandayaver december 31 2017do you hear yourself talk when you have to include that many people that hate you you have to wonder why do the they all hate me alan sandoval alansandoval13 december 31 2017who uses the word haters in a new years wish marlene marlene399 december 31 2017you can t just say happy new year koren pollitt korencarpenter december 31 2017here s trump s new year s eve tweet from 2016happy new year to all including to my many enemies and those who have fought me and lost so badly they just don t know what to do love donald j trump realdonaldtrump december 31 2016this is nothing new for trump he s been doing this for yearstrump has directed messages to his enemies and haters for new year s easter thanksgiving and the anniversary of 911 pictwittercom4fpae2kypa daniel dale ddale8 december 31 2017trump s holiday tweets are clearly not presidentialhow long did he work at hallmark before becoming president steven goodine sgoodine december 31 2017he s always been like this the only difference is that in the last few years his filter has been breaking down roy schulze thbthttt december 31 2017who apart from a teenager uses the term haters wendy wendywhistles december 31 2017he s a fucking 5 year old who knows rainyday80 december 31 2017so to all the people who voted for this a hole thinking he would change once he got into power you were wrong 70yearold men don t change and now he s a year olderphoto by andrew burtongetty images,news,"december 31, 2017",fake
1,drunk bragging trump staffer started russian collusion investigation,house intelligence committee chairman devin nunes is going to have a bad day he s been under the assumption like many of us that the christopher steeledossier was what prompted the russia investigation so he s been lashing out at the department of justice and the fbi in order to protect trump as it happens the dossier is not what started the investigation according to documents obtained by the new york timesformer trump campaign adviser george papadopoulos was drunk in a wine bar when he revealed knowledge of russian opposition research on hillary clintonon top of that papadopoulos wasn t just a covfefe boy for trump as his administration has alleged he had a much larger role but none so damning as being a drunken fool in a wine bar coffee boys don t help to arrange a new york meeting between trump and president abdel fattah elsisi of egypt two months before the election it was known before that the former aide set up meetings with world leaders for trump but team trump ran with him being merely a coffee boyin may 2016 p

In [12]:
# To simplify and speed up processing, as submission in this section is to assess ability to learn and apply 
# NLP modelling techniques, I will drop title/subject/date column and only use 'text' and 'True/Fake' column.

# Create a copy of combined_df and delete the unnecessary columns from new dataframe
combined_text_And_TrueFalse_df = combined_df.copy()

del combined_text_And_TrueFalse_df["title"]
del combined_text_And_TrueFalse_df["subject"]
del combined_text_And_TrueFalse_df["date"]

In [13]:
combined_text_And_TrueFalse_df

,text,True/Fake
0,donald trump just couldn t wish all americans a happy new year and leave it at that instead he had to give a shout out to his enemies haters and the very dishonest fake news media the former reality show star had just one job to do and he couldn t do it as our country rapidly grows stronger and smarter i want to wish all of my friends supporters enemies haters and even the very dishonest fake news media a happy and healthy new year president angry pants tweeted 2018 will be a great year for america as our country rapidly grows stronger and smarter i want to wish all of my friends supporters enemies haters and even the very dishonest fake news media a happy and healthy new year 2018 will be a great year for america donald j trump realdonaldtrump december 31 2017trump s tweet went down about as welll as you d expectwhat kind of president sends a new year s greeting like this despicable petty infantile gibberish only trump his lack of decency won t even allow him to rise above the gutter long enough to wish the american citizens a happy new year bishop talbert swan talbertswan december 31 2017no one likes you calvin calvinstowell december 31 2017your impeachment would make 2018 a great year for america but i ll also accept regaining control of congress miranda yaver mirandayaver december 31 2017do you hear yourself talk when you have to include that many people that hate you you have to wonder why do the they all hate me alan sandoval alansandoval13 december 31 2017who uses the word haters in a new years wish marlene marlene399 december 31 2017you can t just say happy new year koren pollitt korencarpenter december 31 2017here s trump s new year s eve tweet from 2016happy new year to all including to my many enemies and those who have fought me and lost so badly they just don t know what to do love donald j trump realdonaldtrump december 31 2016this is nothing new for trump he s been doing this for yearstrump has directed messages to his enemies and haters for new year s easter thanksgiving and the anniversary of 911 pictwittercom4fpae2kypa daniel dale ddale8 december 31 2017trump s holiday tweets are clearly not presidentialhow long did he work at hallmark before becoming president steven goodine sgoodine december 31 2017he s always been like this the only difference is that in the last few years his filter has been breaking down roy schulze thbthttt december 31 2017who apart from a teenager uses the term haters wendy wendywhistles december 31 2017he s a fucking 5 year old who knows rainyday80 december 31 2017so to all the people who voted for this a hole thinking he would change once he got into power you were wrong 70yearold men don t change and now he s a year olderphoto by andrew burtongetty images,fake
1,house intelligence committee chairman devin nunes is going to have a bad day he s been under the assumption like many of us that the christopher steeledossier was what prompted the russia investigation so he s been lashing out at the department of justice and the fbi in order to protect trump as it happens the dossier is not what started the investigation according to documents obtained by the new york timesformer trump campaign adviser george papadopoulos was drunk in a wine bar when he revealed knowledge of russian opposition research on hillary clintonon top of that papadopoulos wasn t just a covfefe boy for trump as his administration has alleged he had a much larger role but none so damning as being a drunken fool in a wine bar coffee boys don t help to arrange a new york meeting between trump and president abdel fattah elsisi of egypt two months before the election it was known before that the former aide set up meetings with world leaders for trump but team trump ran with him being merely a coffee boyin may 2016 papadopoulos revealed to australian diplomat alexander downer that russian officials were shopping around possible dirt on thendemocratic presidential nominee hillary clinton exactly how much

In [14]:
# Check if any null value in dataframe, will have to delete if so
combined_text_And_TrueFalse_df.isnull().values.any()

False

In [15]:
# Drop any duplicate row of data for 'text' column, as it may negatively affect the modelling process
combined_text_And_TrueFalse_df = combined_text_And_TrueFalse_df.drop_duplicates('text')

In [16]:
combined_text_And_TrueFalse_df

,text,True/Fake
0,donald trump just couldn t wish all americans a happy new year and leave it at that instead he had to give a shout out to his enemies haters and the very dishonest fake news media the former reality show star had just one job to do and he couldn t do it as our country rapidly grows stronger and smarter i want to wish all of my friends supporters enemies haters and even the very dishonest fake news media a happy and healthy new year president angry pants tweeted 2018 will be a great year for america as our country rapidly grows stronger and smarter i want to wish all of my friends supporters enemies haters and even the very dishonest fake news media a happy and healthy new year 2018 will be a great year for america donald j trump realdonaldtrump december 31 2017trump s tweet went down about as welll as you d expectwhat kind of president sends a new year s greeting like this despicable petty infantile gibberish only trump his lack of decency won t even allow him to rise above the gutter long enough to wish the american citizens a happy new year bishop talbert swan talbertswan december 31 2017no one likes you calvin calvinstowell december 31 2017your impeachment would make 2018 a great year for america but i ll also accept regaining control of congress miranda yaver mirandayaver december 31 2017do you hear yourself talk when you have to include that many people that hate you you have to wonder why do the they all hate me alan sandoval alansandoval13 december 31 2017who uses the word haters in a new years wish marlene marlene399 december 31 2017you can t just say happy new year koren pollitt korencarpenter december 31 2017here s trump s new year s eve tweet from 2016happy new year to all including to my many enemies and those who have fought me and lost so badly they just don t know what to do love donald j trump realdonaldtrump december 31 2016this is nothing new for trump he s been doing this for yearstrump has directed messages to his enemies and haters for new year s easter thanksgiving and the anniversary of 911 pictwittercom4fpae2kypa daniel dale ddale8 december 31 2017trump s holiday tweets are clearly not presidentialhow long did he work at hallmark before becoming president steven goodine sgoodine december 31 2017he s always been like this the only difference is that in the last few years his filter has been breaking down roy schulze thbthttt december 31 2017who apart from a teenager uses the term haters wendy wendywhistles december 31 2017he s a fucking 5 year old who knows rainyday80 december 31 2017so to all the people who voted for this a hole thinking he would change once he got into power you were wrong 70yearold men don t change and now he s a year olderphoto by andrew burtongetty images,fake
1,house intelligence committee chairman devin nunes is going to have a bad day he s been under the assumption like many of us that the christopher steeledossier was what prompted the russia investigation so he s been lashing out at the department of justice and the fbi in order to protect trump as it happens the dossier is not what started the investigation according to documents obtained by the new york timesformer trump campaign adviser george papadopoulos was drunk in a wine bar when he revealed knowledge of russian opposition research on hillary clintonon top of that papadopoulos wasn t just a covfefe boy for trump as his administration has alleged he had a much larger role but none so damning as being a drunken fool in a wine bar coffee boys don t help to arrange a new york meeting between trump and president abdel fattah elsisi of egypt two months before the election it was known before that the former aide set up meetings with world leaders for trump but team trump ran with him being merely a coffee boyin may 2016 papadopoulos revealed to australian diplomat alexander downer that russian officials were shopping around possible dirt on thendemocratic presidential nominee hillary clinton exactly how much

In [17]:
# Split the data equally into train and test set, shuffle them
train, test = train_test_split(combined_text_And_TrueFalse_df, test_size=0.5, random_state=42, shuffle=True)

In [18]:
test

text  \
42133                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [19]:
# With reference from:
# https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a
# Naive Bayes using count vectorizer and TF-IDF

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

NBPipe = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
                    ])

# Train NB model
NBPipe = NBPipe.fit(train.text, train["True/Fake"])

In [20]:
# Do prediction based on trained model
import numpy as np

NBPredicted = NBPipe.predict(test.text)

In [21]:
# Find accuracy for NB model
from sklearn.metrics import accuracy_score

NBAccuracy = accuracy_score(test["True/Fake"], NBPredicted)
NBAccuracy

0.9257361693318843

In [22]:
# Find F1 score for NB model
from sklearn.metrics import f1_score

NBf1 = f1_score(test["True/Fake"], NBPredicted, pos_label="true")
NBf1

0.9358315074006172

In [23]:
# With reference from:
# https://stackoverflow.com/questions/42793709/how-to-optimize-a-sklearn-pipeline-using-xgboost-for-a-different-eval-metric

# Pipeline for XGBoost using count vectorizer and TF-IDF

import xgboost as xgb
from xgboost.sklearn import XGBClassifier

xgboostPipe = Pipeline([('vect', CountVectorizer()),
                        ('tfidf', TfidfTransformer()),
                        ('classifier', XGBClassifier())
                       ])

# Train XGBoost model
xgboostPipe = xgboostPipe.fit(train.text, train["True/Fake"])

C:\Users\Chan\.conda\envs\main\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:02:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [24]:
# Do prediction based on trained model
xgBoostpredicted = xgboostPipe.predict(test.text)

In [25]:
# Find accuracy for XGBoost model
xgBoostaccuracy = accuracy_score(test["True/Fake"], xgBoostpredicted)
xgBoostaccuracy

0.9963256223153755

In [26]:
# Find F1 score for XGBoost model
xgBoostf1 = f1_score(test["True/Fake"], xgBoostpredicted, pos_label="true")
xgBoostf1

0.9966529958044595

In [27]:
# With reference from:
# https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a

# Pipeline for SVM using count vectorizer and TF-IDF

from sklearn.linear_model import SGDClassifier

svmPipe = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf-svm', SGDClassifier())
                   ])

# Train SVM model
svmPipe = svmPipe.fit(train.text, train["True/Fake"])

In [28]:
# Do prediction based on trained model
svmPredicted = svmPipe.predict(test.text)

In [29]:
# Find accuracy for SVM model
svmAccuracy = accuracy_score(test["True/Fake"], svmPredicted)
svmAccuracy

0.9907364280908761

In [30]:
# Find F1 score for SVM model
svmF1 = f1_score(test["True/Fake"], svmPredicted, pos_label="true")
svmF1

0.9915768669709661

# Summary

# TF-IDF
With reference from:
https://machinelearningmastery.com/gentle-introduction-bag-words-model/
https://monkeylearn.com/blog/what-is-tf-idf/#:~:text=TF%2DIDF%20is%20a%20statistical,across%20a%20set%20of%20documents.

I chose TF-IDF for feature engineering as TF-IDF score can be fed to algorithms such as Naive Bayes and Support Vector Machines, greatly improving the results of more basic methods like word counts.

A problem with scoring word frequency is that highly frequent words start to dominate in the document (e.g. larger score), but may not contain as much “informational content” to the model as rarer but perhaps domain specific words.

One approach is to rescale the frequency of words by how often they appear in all documents, so that the scores for frequent words like “the” that are also frequent across all documents are penalized.

This approach to scoring is called Term Frequency – Inverse Document Frequency, or TF-IDF for short, where:
TF-IDF is a statistical measure that evaluates how relevant a word is to a document in a collection of documents. This is done by multiplying two metrics: how many times a word appears in a document, and the inverse document frequency of the word across a set of documents.

It has many uses, most importantly in automated text analysis, and is very useful for scoring words in machine learning algorithms for Natural Language Processing (NLP).

TF-IDF (term frequency-inverse document frequency) was invented for document search and information retrieval. It works by increasing proportionally to the number of times a word appears in a document, but is offset by the number of documents that contain the word. So, words that are common in every document, such as this, what, and if, rank low even though they may appear many times, since they don’t mean much to that document in particular.

# Naive Bayes
With reference from:
https://towardsdatascience.com/text-classification-using-naive-bayes-theory-a-working-example-2ef4b7eb7d5a

I chose Naive Bayes as one of the algorithms as Naive Bayes classifiers have been heavily used for text classification and text analysis machine learning problems.

Naive Bayes classifiers are a collection of classification algorithms based on Bayes’ Theorem. It is not a single algorithm but a family of algorithms where all of them share a common principle, i.e. every pair of features being classified is independent of each other.


With reference from:
https://suatatan.com/posts/sklearn_xgboost_tc/

I chose XGBoost as it can help to predict any kind of data if you have already predicted data before. You can classify any kind of data. It can be used for text classification too.
XGBoost also has been the winning algorithm in a number of recent Kaggle competitions.
XGBoost stands for eXtreme Gradient Boosting and is an implementation of gradient boosting machines that pushes the limits of computing power for boosted trees algorithms as it was built and developed for the sole purpose of model performance and computational speed.

XGBoost is a version of gradient boosted decision tree classifier. In boosting, the trees are built sequentially such that each subsequent tree aims to reduce the errors of the previous tree. These subsequent trees are called base or weak learners. Each of these weak learners contributes some vital information for prediction, enabling the boosting technique to produce a strong learner by effectively combining these weak learners. The power of XGBoost lies in its scalability, which drives fast learning through parallel and distributed computing and offers efficient memory usage.


# SVM

With reference from:
https://scikit-learn.org/stable/modules/svm.html
https://monkeylearn.com/text-classification-support-vector-machines-svm/

I chose SVM because provided we can find vector representations which encode as much information from our texts as possible, we will be able to apply the SVM algorithm to text classification problems and obtain very good results.

Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression and outliers detection.

The advantages of support vector machines are:
-Effective in high dimensional spaces.
-Still effective in cases where number of dimensions is greater than the number of samples.
-Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.
-Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.

The disadvantages of support vector machines include:
-If the number of features is much greater than the number of samples, avoid over-fitting in choosing Kernel functions and regularization term is crucial.
-SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation (see Scores and probabilities, below).



# Conclusion

Naive Bayes model gives an accuracy score of 0.9257361693318843, F1 score of 0.9358315074006172.

XGBoost model gives an accuracy score of 0.9963256223153755, F1 score of 0.9966529958044595.

SVM model gives an accuracy score of 0.9907881798892512, F1 score of 0.9916235294117647.


With respect to the accuracy and F1 score, it appears that XGBoost gives the best result with approximately 99.63% accuracy and F1 score of approximately 99.67%. SVM model gives a close result and Naive Bayes is the least performing amongst the three.